# Number 3

There are several cost functions available, but one which is in our interests and is also most widely used in regression and classification tasks is the MSE Cost Function. Your task is to find a polynomial regression model to establish an optimal relation between salary and position level by minimising the MSE cost function using a gradient descent approach. Please ensure the code is written in a Python notebook and put your relevant explanations/comments within the markdown cells. You may use a dataset taken from raw.githubusercontent.com/mkgurucharan/Regression/master/PositionSalaries_Data.csv

## Importing Libraries

These libraries below are used to work with polynomial regression models. In this case, I employed Artificial Neural Network regression to working with the minimizing the MSE cost function.

In [70]:
import pandas;
import numpy;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split;
from tensorflow.keras.layers import Dense, InputLayer;
from tensorflow.keras.models import Sequential;
from tensorflow.keras.utils import plot_model;
from tensorflow.keras.optimizers import Adam;
import matplotlib.pyplot as plt;
from tensorflow.keras.callbacks import EarlyStopping;
import tensorflow

from tensorflow.keras.losses import MAE, MSE

from tqdm import tqdm

## Data Loading & Exploration

This section is about data loading and explorations.  

In [22]:
dataframe = pandas.read_csv("PositionSalaries_Data.csv", index_col = None, );
dataframe.reset_index(drop=True, inplace=True);

# Randomize the data within dataframe
dataframe = dataframe.sample(frac = 1).reset_index(drop = True)

# Print some data
dataframe.head()

,Position,Level,Salary
0,C-level,9,500000
1,Country Manager,5,110000
2,Junior Consultant,2,50000
3,Region Manager,6,150000
4,Senior Consultant,3,60000


In [23]:
# Found that this data contains 10 rows and 3 columns. 
dataframe.shape

(10, 3)

## Data Pre-Processing

This section is about data pre-processing. The following actions are taken to clean and ease the training process:

1. Setting up Features and Label
Position and Level are used as features, while Salary for label

2. Drop Position column
Since level contains flag values for position, the "Position" column should be dropped.

3. Standardize the data with standard scaler
The question is asking me to control the MSE cost. Well, controlling MSE is hard since its barely readable especially when it comes to Gradient Descent. So the plan is to normalize the each data within feature and label, by lowering the standard deviation, in which turning the means within the data into 0.

4. Train-Test Split
Split the data into training and testing with ratio below:

| Split | Ratio |
|---|---|
| Train | 70% |
| Test | 30% |

In [24]:
# Call out Step 1

features = dataframe.drop(columns = ["Salary"]);
label = dataframe["Salary"];

label.shape

(10,)

In [25]:
# Call out Step 2

features = features.drop(columns = ["Position"]);
features.shape

(10, 1)

In [26]:
# Call out Step 3
scaler = StandardScaler();
features = scaler.fit_transform(features);
label = scaler.fit_transform(numpy.array(label).reshape(-1, 1));

features.shape

(10, 1)

In [27]:
feature_train, feature_test, label_train, label_test = train_test_split(
    features, 
    label, 
    train_size = 0.7, 
    test_size = 0.3, 
    random_state = 42
);

## Techniques

In endavour producing a high precision prediction polynomial regression model, the Artificial Neural Network (ANN) algorithm with Adam optimizer gradient descent method is employed. 
The Techniques section are consist of the following steps below:

1. ANN Model Factory
This part contains required meta data, and code design pattern to ease the training process of ANN model

2. Models fitting and evaluation
This part contains the training and evaluation process of the ANN model. Things to be known include:
- Mean Squared Error (MSE) is employed as model's loss (in this question wise, MSE treated as cost) function
- Mean Absolute Error (MAE) is employed as model's metrics function due to easier estimation and its readibility.

Both of MSE and MAE can be qualitatively said as:
"The more near zero both MSE and MAE, the more accurate the model prediction."

### Model Factory

Click [here to open LightGBM docs reference](https://lightgbm.readthedocs.io/en/latest/Parameters.html)

In [72]:
"""
ANNModel

This class implements a OO-pattern to ease model generation. 
"""
class ANNModel:

    def __init__(self):
        # The constructor only allowed to construct a standardized unchangeable configuration.
        model = Sequential();
        
        # Input Layer
        model.add(InputLayer(input_shape = (1,)));
        
        # Hidden Layer

        for i in [self.hidden_layer()]:
            for j in i:
                model.add(j);

        # Output Layer
        model.add(Dense(1, activation = "linear"));
    
        self.model = model;

    def hidden_layer(self):
        n = 1;
        return [
            Dense(32, activation = "relu"),
        ];

    # Early stopping after loss are not improved for some epochs
    def _callback_early_stopping(self):
        early_stopping_tolerance = 50;

        return EarlyStopping(
            monitor = "val_loss",
            patience = early_stopping_tolerance,
            restore_best_weights = True,
        );

    def fitting(self, train_feature, train_label, test_feature, test_label):
        
        model = self.model;
        model.compile(loss = "mse", optimizer = Adam(learning_rate = 1e-3), metrics = ["mae"])

        plot_model(model, to_file = ("model_architecture.png"), show_shapes = True);

        optimizer = Adam(learning_rate = 1e-3)

        for epoch in range(0, 1000):
            with tensorflow.GradientTape() as tape:
                logits = model(train_feature, training = True)

                loss = MSE(train_label, logits)
                metric = MAE(train_label, logits)

            gradient = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradient, model.trainable_weights))

            # Validation
            val_logits = model(test_feature, training = False)

            val_loss = MSE(test_label, val_logits)
            val_metric = MAE(test_label, val_logits)

            print(f"Epoch {(epoch+1)} of 1000 | loss (mse): {tensorflow.reduce_mean(loss).numpy()} - mae: {tensorflow.reduce_mean(val_metric).numpy()} - val loss: {tensorflow.reduce_mean(val_loss).numpy()} - val mae: {tensorflow.reduce_mean(metric).numpy()}")
            
        return model;

In [73]:
model = ANNModel();
dojo = model.fitting(feature_train, label_train, feature_test, label_test);
loss, accuracy = dojo.evaluate(feature_test, label_test);

You must install pydot (`pip install pydot`) for `plot_model` to work.
Epoch 1 of 1000 | loss (mse): 1.688189148902893 - mae: 0.38700070977211 - val loss: 0.2071644812822342 - val mae: 0.9579175114631653
Epoch 2 of 1000 | loss (mse): 1.665553331375122 - mae: 0.38476309180259705 - val loss: 0.20157186686992645 - val mae: 0.9498056769371033
Epoch 3 of 1000 | loss (mse): 1.643141746520996 - mae: 0.38252201676368713 - val loss: 0.196127250790596 - val mae: 0.941693902015686
Epoch 4 of 1000 | loss (mse): 1.6209551095962524 - mae: 0.380278617143631 - val loss: 0.19083206355571747 - val mae: 0.9335829019546509
Epoch 5 of 1000 | loss (mse): 1.5989927053451538 - mae: 0.37803375720977783 - val loss: 0.18568718433380127 - val mae: 0.9254695177078247
Epoch 6 of 1000 | loss (mse): 1.5772536993026733 - mae: 0.3758814334869385 - val loss: 0.18077968060970306 - val mae: 0.9173527956008911
Epoch 7 of 1000 | loss (mse): 1.5557414293289185 - mae: 0.37374213337898254 - val loss: 0.17603468894958496 - val 